In [11]:
import os
os.chdir("../Functionality/")
from implicit.nearest_neighbours import bm25_weight
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix
import pandas as pd
import spotipy
import sys
from spotipy.oauth2 import SpotifyClientCredentials

import numpy as np
from ipynb.fs.full.Model import *
from ipynb.fs.full.Spotify_Functions import *
from ipynb.fs.full.Load_Starting_Files import *
import warnings
warnings.filterwarnings('ignore')

In [12]:
client_id = 'e42bc8c7773341c98c22d1099ec06d9b'
client_secret = 'a1441373650e44979c617106a84352e3'

spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id,
                                                                client_secret=client_secret))

# Test Echo Chamber Performance UK

In [13]:
def get_predictions(data, userid, user_history, unique_tracks, model, user_plays):
    
    track_ids, scores = model.recommend(userid, user_plays[userid], N=10, filter_already_liked_items=False)

    prediction_data = unique_tracks[unique_tracks['trackID'].isin(track_ids)]
    
    prediction_data['confidence'] = scores
    
    prediction_data['track_liked'] = prediction_data['recordingMBID'].isin(user_history['recordingMBID'])
      
    prediction_data['artist_liked'] = prediction_data['artist-MBID'].isin(user_history['artist-MBID'])
 
    return(prediction_data)

In [14]:
def get_similar_artist(prediction, user_history):
    results = []
    artists = prediction[prediction['artist_liked'] == False]['artistName']
    if len(artists) > 0:
        for name in artists:
            artist_id = lookup_artist(spotify, str(name))
            if artist_id != None:
                similar_artists = get_related_artist(spotify, artist_id)
                results.append(any(item in user_history['artistName'].values for item in similar_artists))
    

    return(results)

In [15]:
def test_model(data, model, user_plays, sample_ids, unique_tracks):

    
    results = []
    for userid in sample_ids:
        
        user_history = data[data['userID'] == userid]
        
        user_prediction = get_predictions(data, userid, user_history, unique_tracks, model, user_plays)
        
        results.append((user_prediction["track_liked"].sum(), user_prediction["artist_liked"].sum(), get_similar_artist(user_prediction, user_history)))

        
        
    return(results)

        
        
        

# Test ALS factors

# Test Change in Alpha

In [16]:
def test_alpha(data, unique_tracks, sample_ids):
    results = []
    test_alpha_table = [1.0,2.0,3.0,5.0,10.0]
    for test in test_alpha_table:
        model, user_plays = create_model(data[['userID','trackID','count']], alpha = test)
        results.append(test_model(data, model, user_plays, sample_ids, unique_tracks))
    return(results)

In [17]:
def perform_test(data, test):
    unique_tracks = data.drop_duplicates(subset = ["trackID"])
    unique_tracks = unique_tracks[['trackID','recordingMBID', 'trackName', 'artist-MBID', 'artistName']]
    sample_ids = np.random.choice(data['userID'], 100)
    results = test(data, unique_tracks, sample_ids) 
    return(results)

In [27]:
#uk_data = pd.read_pickle('../../cleaned_data/uk_data.pkl')
#uk_results = perform_test(uk_data, test_alpha)
us_data = pd.read_pickle('../../cleaned_data/us_data.pkl')
us_results = perform_test(us_data, test_alpha)
#pl_data = pd.read_pickle('../../cleaned_data/pl_data.pkl')
#pl_results = perform_test(pl_data, test_alpha)


In [23]:
def mean_results(results):
    full_results = []
    for result_set in results:
        average_already_liked, average_artist_liked, similar_artists = [],[],[]
        for result in result_set:
            (already_liked, artist_liked, similar_artist) = result
            average_already_liked.append(already_liked)
            average_artist_liked.append(artist_liked)
            similar_artists.append(sum(similar_artist)+artist_liked)
        full_results.append([np.mean(average_already_liked), np.mean(average_artist_liked), np.mean(similar_artists)])
    return(full_results)

In [28]:
#uk_full_results = mean_results(uk_results)
us_full_results = mean_results(us_results)
#pl_full_results = mean_results(pl_results)

In [21]:
uk_full_results

[[1.34, 3.86, 6.82],
 [1.34, 3.87, 6.83],
 [1.34, 3.86, 6.82],
 [1.34, 3.86, 6.82],
 [1.34, 3.85, 6.82]]

In [29]:
us_full_results

[[1.81, 4.25, 7.45],
 [1.81, 4.25, 7.44],
 [1.82, 4.26, 7.44],
 [1.81, 4.25, 7.45],
 [1.81, 4.25, 7.45]]

In [25]:
pl_full_results

[[1.18, 3.72, 6.19],
 [1.18, 3.72, 6.19],
 [1.18, 3.72, 6.19],
 [1.18, 3.71, 6.19],
 [1.18, 3.73, 6.22]]

In [ ]:
#uk_df_model_op = pd.DataFrame(uk_full_results)

In [ ]:
#us_df_model_op = pd.read_csv('../../cleaned_data/us_df_model_op.csv', index_col = 0)

In [ ]:
#us_df_model_op.columns = (["Average track previously liked","Average artist previously liked", "Average similar artist liked"])
#us_df_model_op.loc[3]